In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
from astropy.io import ascii
from astropy.table import Table, vstack
from scipy import stats

In [2]:
names = ['Aadit', 'Adam', 'Annette', 'Diamanta', 'Isaac', 'Isobel', 'Jay', 'Joceline', 'Keith', 'Meredith', 'Regina', 'Ross', 'Scott', 'Thaddeus']

In [4]:
abroad = ascii.read("abroad.csv", names=names)
home = ascii.read("hrishi_votes.csv", names=names)
all_data = vstack([home, abroad])

In [5]:
#to remove "No Vote","No",""(blank) and replace them with "15"(lower than the lowest possible preference)

def strip_strings(x):
    for index in range(0,len(x)):
        for second_index in range(0,len(x[0])):
            if "=>" in x[index][second_index]:
                x[index][second_index] = (x[index][second_index]).split("=>")[1].replace('"',"")
            
    for index in range(0,len(x)):
        for second_index in range(0,len(x[0])):
            if (x[index][second_index]=="NO VOTE")|(x[index][second_index]=="")|(x[index][second_index]=="NO"):
                x[index][second_index]= int(15)
            else:
                x[index][second_index]= int(x[index][second_index])

In [6]:
strip_strings(all_data)

In [7]:
from __future__ import print_function
import numpy as np
from itertools import combinations, permutations
from lp_solve import lp_solve
import csv

In [10]:
def kendalltau_dist(rank_a, rank_b):
    tau = 0
    n_candidates = len(rank_a)
    for i, j in combinations(range(n_candidates), 2):
        tau += (np.sign(rank_a[i] - rank_a[j]) ==
                -np.sign(rank_b[i] - rank_b[j]))
    return tau

#graph-distance conversion of ranks
def _build_graph(ranks):
    n_voters, n_candidates = ranks.shape
    edge_weights = np.zeros((n_candidates, n_candidates))
    for i, j in combinations(range(n_candidates), 2):
        preference = ranks[:, i] - ranks[:, j]
        h_ij = np.sum(preference < 0)  # prefers i to j
        h_ji = np.sum(preference > 0)  # prefers j to i
        if h_ij > h_ji:
            edge_weights[i, j] = h_ij - h_ji
        elif h_ij < h_ji:
            edge_weights[j, i] = h_ji - h_ij
    return edge_weights

#function that actually calculates k-y ranks by implementing fancy integer programming
def rankaggr_lp(ranks):
    """Kemeny-Young optimal rank aggregation"""

    n_voters, n_candidates = ranks.shape
    
    # maximize c.T * x
    edge_weights = _build_graph(ranks)
    c = -1 * edge_weights.ravel()  

    idx = lambda i, j: n_candidates * i + j

    # constraints for every pair
    pairwise_constraints = np.zeros(((n_candidates * (n_candidates - 1)) / 2,
                                     n_candidates ** 2))
    for row, (i, j) in zip(pairwise_constraints,
                           combinations(range(n_candidates), 2)):
        row[[idx(i, j), idx(j, i)]] = 1

    # and for every cycle of length 3
    triangle_constraints = np.zeros(((n_candidates * (n_candidates - 1) *
                                     (n_candidates - 2)),
                                     n_candidates ** 2))
    for row, (i, j, k) in zip(triangle_constraints,
                              permutations(range(n_candidates), 3)):
        row[[idx(i, j), idx(j, k), idx(k, i)]] = 1

    constraints = np.vstack([pairwise_constraints, triangle_constraints])
    constraint_rhs = np.hstack([np.ones(len(pairwise_constraints)),
                                np.ones(len(triangle_constraints))])
    constraint_signs = np.hstack([np.zeros(len(pairwise_constraints)),  # ==
                                  np.ones(len(triangle_constraints))])  # >=

    obj, x, duals = lp_solve(c, constraints, constraint_rhs, constraint_signs,
                             xint=range(1, 1 + n_candidates ** 2))

    x = np.array(x).reshape((n_candidates, n_candidates))
    aggr_rank = x.sum(axis=1)

    return obj, aggr_rank

In [12]:
import csv
ascii.write(all_data, 'values_hrishi.dat')

with open('test_file_hrishi.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    [writer.writerow(r) for r in ascii.read('values_hrishi.dat')]
    
with open('test_file_hrishi.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    table = [[int(e) for e in r] for r in reader]

In [13]:
#and now to the YNC case!

names = ['Aadit', 'Adam', 'Annette', 'Diamanta', 'Isaac', 'Isobel', 'Jay', 'Joceline', 'Keith',\
         'Meredith', 'Regina', 'Ross', 'Scott', 'Thaddeus']

with open('test_file_hrishi.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    prefs = np.array([[int(e) for e in r] for r in reader])

#this gives the optimal ranking

_, aggr = rankaggr_lp(prefs)
score = np.sum(kendalltau_dist(aggr, rank) for rank in prefs)
print("A Kemeny-Young aggregation with score {} is: {}".format(
    score,
    ", ".join(names[i] for i in np.argsort(aggr))))

A Kemeny-Young aggregation with score 7785 is: Regina, Joceline, Annette, Meredith, Jay, Isobel, Diamanta, Isaac, Scott, Ross, Keith, Adam, Aadit, Thaddeus


C:\Anaconda\lib\site-packages\lp_solve.py:38: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if f == None:
